In [ ]:
import numpy as np
import scipy as sy
import pandas as pd
import matplotlib.pyplot as plt
import time
import sys
import os
import ast
import json
import copy 
import itertools
from itertools import islice
from tqdm import *
from math import ceil
from collections import defaultdict
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import roc_auc_score, accuracy_score
from scipy.sparse import csr_matrix, dok_matrix
from ipynb.fs.full.Random_Sample_Mapper import *

# Read In Data

In [ ]:
notebook_path = os.path.abspath("BPR_OPT_Binary_Model.ipynb")
users_items_file_path = os.path.join(os.path.dirname(notebook_path), "data/australian_users_items.json")
items_file_path = os.path.join(os.path.dirname(notebook_path), "data/items_meta_data.json")
users_meta_data_file_path = os.path.join(os.path.dirname(notebook_path), "data/users_meta_data.json")

In [ ]:
users_items = []
with open(users_items_file_path, 'r') as data:
    for line in data:
        users_items.append(ast.literal_eval(line))

In [ ]:
with open(items_file_path, 'r') as data:
    games_dict = json.load(data)

In [ ]:
with open(users_meta_data_file_path, 'r') as file:
    users_meta_data = json.load(file)

# Process Data into Training and Testing Sets

Using default dict for efficient data retrieval for users-items playtime relationship

In [ ]:
usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)
playtimesPerItem = defaultdict(dict)
playtimesPerUser = defaultdict(dict)
itemNames = defaultdict(str)

In [ ]:
train_usersPerItem = defaultdict(set)
train_itemsPerUser = defaultdict(set)
test_usersPerItem = defaultdict(set)
test_itemsPerUser = defaultdict(set)

In [ ]:
for game in games_dict:
    if 'owners' in games_dict[game]:
        usersPerItem[game] = set(games_dict[game]['owners'].keys())

In [ ]:
for user in users_items:
    u_id = user['user_id']
    items = [item['item_id'] for item in user['items']]
    itemsPerUser[u_id] = items
    
    items_train, items_test = train_test_split(items, test_size = 0.20)
    train_itemsPerUser[u_id] = items_train
    test_itemsPerUser[u_id] = items_test

    playtimesPerUser[user['user_id']] = dict((item['item_id'], item['playtime_forever']) for item in user['items'])
    for item in user['items']:
        itemNames[item['item_id']] = item['item_name']
        playtimesPerItem[item['item_id']][user['user_id']] = item['playtime_forever']

In [ ]:
for user in train_itemsPerUser:
    for item in train_itemsPerUser[user]:
        train_usersPerItem[item].add(user)
        
    for item in test_itemsPerUser[user]:
        test_usersPerItem[item].add(user)

## Approach 1: Scheduled Sampling with Map-Reduce Algorithm

In [ ]:
nUsers = len(itemsPerUser)
nItems = len(usersPerItem)
users = list(itemsPerUser.keys())
items = list(usersPerItem.keys())

In [ ]:
train_user_item_counts = dict((k, len(v)) for k, v in train_itemsPerUser.items())

In [ ]:
test_user_item_counts = dict((k, len(v)) for k, v in test_itemsPerUser.items())

In [ ]:
train_datafile = 'data/train_sample_in.tsv'
train_mapout1 = 'data/train_sample_map1.tsv'
train_mapout2 = 'data/train_sample_map2.tsv'
train_outfile = 'data/train_sample_out.tsv'

f = open(train_datafile,'w')
for u, its in train_itemsPerUser.items():
    for i in its:
        print(default_formatter(u,i), file=f)
f.close()

In [ ]:
# run two stages of mapreduce
train_mapper = Mapper(train_user_item_counts)
mapreduce(train_datafile, train_mapout1, mapper=train_mapper, reducer=reducer)
mapreduce(train_datafile, train_mapout2, mapper=indicator_mapper)  # map the data again
mapreduce([train_mapout1, train_mapout2], train_outfile, reducer=indicator_reducer)

In [ ]:
def generate_test_sample():
    test_sample = []
    for u, its in test_itemsPerUser.items():
        for i in its:
            j = random.choice(items)
            test_sample.append((u, i, j))
    return test_sample

In [ ]:
def trim(u, i, j):
    return u[1:len(u)-1], i[2:len(i)-2], j[1:len(j)-2]

In [ ]:
class ExternalSchedule(object):

    def __init__(self, filepath):
        self.filepath = filepath
        
    def generate_random_samples(self, size=None, rand=True):
        f = open(self.filepath)
        samples = [map(str, line.strip().split()) for line in f]
        if rand:
            random.shuffle(samples)  # important!
        if size is None:
            size = len(samples)
        for u, i, j in samples[:size]:
            yield u, i, j
            
    def generate_list_samples(self):
        f = open(self.filepath)
        samples = [map(str, line.strip().split()) for line in f]
        triplets = []
        for u, i, j in samples:
            u, i, j = trim(u, i, j)
            triplets.append((u, i, j))
            
        return triplets

In [ ]:
sampler = ExternalSchedule(train_outfile)  # schedule is one-indexed

In [ ]:
train_sample = sampler.generate_list_samples()

In [ ]:
len(train_sample) * 0.05

# Utility Functions

In [ ]:
def inner(x, y):
    return sum([a*b for a,b in zip(x,y)])

In [ ]:
def binary_label(u, i, j):
    c = Counter(itemsPerUser[u])
    if c[i] > c[j]:
        return 1
    else:
        return 0

In [ ]:
def generate_outputs(sample):
    predictions = []
    labels = []
    for u, i, j in sample:
        predict = sigmoid(prediction(u, i, j))
        label = binary_label(u, i, j)
        predictions.append(predict)
        labels.append(label)
            
    return np.rint(predictions), labels

In [ ]:
def accuracy(predictions, labels):
    differences = [1 if x == y else 0 for x, y in zip(predictions, labels)]
    return sum(differences) / len(differences)

## Sigmoid Function

\begin{equation*}
\sigma(x) = \frac{1}{1 + e^{-x}}
\end{equation*}

In [ ]:
def sigmoid(x):
    #Numerically stable sigmoid function.
    #Taken from: https://timvieira.github.io/blog/post/2014/02/11/exp-normalize-trick/
    if x >= 0:
        z = np.exp(-x)
        return 1 / (1 + z)
    else:
        # if x is less than zero then z will be small, denom can't be
        # zero because it's 1+z.
        z = np.exp(x)
        return z / (1 + z)

# Simple (Biase Only) Latent Factor Model with Binary Classification

In [ ]:
itemBiases = defaultdict(float)

In [ ]:
def unpack(theta):
    global itemBiases
    itemBiases = dict(zip(items, theta))

## Prediction Function

\begin{equation*}
f(i, j) = \beta_i - \beta_j
\end{equation*}

\begin{equation*}
p(i >_u j) = \sigma(f(i, j))
\end{equation*}

In [ ]:
def prediction(u, item_i, item_j):
    return itemBiases[item_i] - itemBiases[item_j]

\begin{equation*}
\text{BPR-OPT} := \text{argmax} \ln(\sigma(\beta_i - \beta_j))
\end{equation*}

\begin{equation*}
\text{Cost Function}:= \sum_{u,i,j} ln\left( \frac{1}{1 + e^{\beta_j - \beta_i}} \right)
\end{equation*}

## Cost Function

In [ ]:
def cost(theta):
    unpack(theta)
    cost = 0.0
    predictions = []
    labels = []
    for u, i, j in sampler.generate_random_samples(size=70000):
        u, i, j = trim(u, i ,j)
        x = prediction(u, i, j)
        
        predictions.append(sigmoid(x))
        labels.append(binary_label(u, i, j))

        cost += np.log(sigmoid(x))
        
    print('Current Cost: %s' % -cost)
    print('Current Accuracy: %s' % accuracy(np.rint(predictions), labels))
    print('-------------------------------------------------------------------')
    
    return -cost

\begin{equation*}
\frac{\partial }{\partial x} ln\sigma(x) = \frac{1}{1 + e^x} = \sigma(-x)
\end{equation*}

## Partial Derivatives

\begin{equation*}
\frac{\partial f}{\partial \beta_i} = \frac{e^{\beta_j - \beta_i}}{1 + e^{\beta_j - \beta_i}} = \frac{1}{1 + e^{\beta_i - \beta_j}}
\end{equation*}

\begin{equation*}
\frac{\partial f}{\partial \beta_j} = -\frac{e^{\beta_j - \beta_i}}{1 + e^{\beta_j - \beta_i}} = -\frac{1}{1 + e^{\beta_i - \beta_j}}
\end{equation*}

In [ ]:
def derivative(theta):
    unpack(theta)
    dItemBiases = defaultdict(float)
    for u, i, j in sampler.generate_random_samples(size=70000):
        u, i, j = trim(u, i ,j)
        x = prediction(u, i, j)
        dbase = -1 / (1 + np.exp(x)) # negative gradient descent for maximizing
        dItemBiases[i] += dbase
        dItemBiases[j] -= dbase
    dtheta = [dItemBiases[i] for i in items]
    return np.array(dtheta)

In [ ]:
res, f, d = sy.optimize.fmin_l_bfgs_b(cost, [0.0]*nItems, derivative)

In [ ]:
d

In [ ]:
unpack(res)

In [ ]:
test_sample = generate_test_sample()

In [ ]:
predictions, labels = generate_outputs(test_sample)

In [ ]:
accuracy(predictions, labels)

# Complete Latent Factor Model with Binary Classification

A simple non-biased latent factor model that is wrapped into a binary function (sigmoid function) as a base line model, using popularity as the item's sole feature

In [ ]:
itemBiases = defaultdict(float)

For each user and item we now have a low dimensional descriptor (representing that user's preferences, and that item's properties), of dimension K.

In [ ]:
userGamma = {}
itemGamma = {}

In [ ]:
K = 2

In [ ]:
for u in itemsPerUser:
    userGamma[u] = [random.random() * 0.1 - 0.05 for k in range(K)]

In [ ]:
for i in usersPerItem:
    itemGamma[i] = [random.random() * 0.1 - 0.05 for k in range(K)]

We'll use another library in this example to perform gradient descent. This library requires that we pass it a "flat" parameter vector (theta) containing all of our parameters. This utility function just converts between a flat feature vector, and our model parameters, i.e., it "unpacks" theta into our offset and bias parameters.

In [ ]:
def unpack(theta):
    global itemBiases
    global userGamma
    global itemGamma
    index = 0
    itemBiases = dict(zip(items, theta[0:index + nItems]))
    index += nItems
    for u in users:
        userGamma[u] = theta[index:index + K]
        index += K
    for i in items:
        itemGamma[i] = theta[index:index + K]
        index += K

## Prediction Function

\begin{equation*}
f(u, i, j) = \gamma_u \gamma_i + \beta_i - (\gamma_u \gamma_j + \beta_j)
\end{equation*}

\begin{equation*}
p(i >_u j) = \sigma(f(u, i, j))
\end{equation*}

In [ ]:
def prediction(user, item_i, item_j):
    return inner(userGamma[user], itemGamma[item_i]) + itemBiases[item_i] - (inner(userGamma[user], itemGamma[item_j]) + itemBiases[item_j]) 

## Cost Function

\begin{equation*}
\text{BPR-OPT} := \text{argmax} \ln(\sigma(\gamma_u \gamma_i + \beta_i - (\gamma_u \gamma_j + \beta_j)))
\end{equation*}

\begin{equation*}
\text{Cost Function}:= \sum_{u,i,j} ln\left( \frac{1}{1 + e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}} \right)
\end{equation*}

In [ ]:
def cost(theta):
    unpack(theta)
    cost = 0.0
    predictions = []
    for u, i, j in sampler.generate_random_samples(size=5000, rand=False):
        u, i, j = trim(u, i ,j)
        x = prediction(u, i, j)
        
        predictions.append(sigmoid(x))
        labels.append(binary_label(u, i, j))

        cost += np.log(sigmoid(x))
        
    print('Current Cost: %s' % -cost)
    print('Current Accuracy: %s' % accuracy(predictions, labels))
    print('-------------------------------------------------------------------')
        
    return -cost

## Partial Derivatives

\begin{equation*}
\frac{\partial f}{\partial \gamma_{u,k}} = \frac{(\gamma_{i,k} - \gamma_{j,k}) \cdot e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}{1 + e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}
\end{equation*}

\begin{equation*}
\frac{\partial f}{\partial \gamma_{i,k}} = \frac{\gamma_{u,k} \cdot e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}{1 + e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}
\end{equation*}

\begin{equation*}
\frac{\partial f}{\partial \gamma_{j,k}} = -\frac{\gamma_{u,k} \cdot e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}{1 + e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}
\end{equation*}

\begin{equation*}
\frac{\partial f}{\partial \beta_i} = \frac{e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}{1 + e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}
\end{equation*}

\begin{equation*}
\frac{\partial f}{\partial \beta_j} = -\frac{e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}{1 + e^{\gamma_u \gamma_j + \beta_j - (\gamma_u \gamma_i + \beta_i)}}
\end{equation*}

In [ ]:
def derivative(theta):
    unpack(theta)
    dItemBiases = defaultdict(float)
    dUserGamma = {}
    dItemGamma = {}
    for u in users:
        dUserGamma[u] = [0.0 for k in range(K)]
    for i in items:
        dItemGamma[i] = [0.0 for k in range(K)]
    for u, i, j in sampler.generate_random_samples(size=5000, rand=False):
        u, i, j = trim(u, i ,j)
        x = prediction(u, i ,j)
        dbase = -1 / (1 + np.exp(x)) # negative gradient descent for maximizing
        dItemBiases[i] += dbase
        dItemBiases[j] -= dbase
        for k in range(K):
            dUserGamma[u][k] += (itemGamma[i][k] - itemGamma[j][k]) * dbase
            dItemGamma_k = userGamma[u][k] * dbase
            dItemGamma[i][k] += dItemGamma_k
            dItemGamma[j][k] -= dItemGamma_k
    dtheta = [dItemBiases[i] for i in items]
    for u in users:
        dtheta += dUserGamma[u]
    for i in items:
        dtheta += dItemGamma[i]
    return np.array(dtheta)

In [ ]:
complete_res, complete_f, complete_d = sy.optimize.fmin_l_bfgs_b(cost, 
                                [0.0]*nItems + # Initialize beta
                                [random.random() * 0.1 - 0.05 for k in range(K*(nUsers + nItems))], # Gamma
                                derivative)

In [ ]:
complete_d

In [ ]:
unpack(complete_res)

In [ ]:
predictions, labels = generate_outputs(train_sample[:1000])

In [ ]:
accuracy(predictions, labels)

In [ ]:
predictions, labels = generate_outputs(test_sample)

In [ ]:
accuracy(predictions, labels)